In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
# import dynamo as dyn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import spateo as st

sys.path.insert(0, "/home/ylu/project")
from utils import *
from tqdm import tqdm

2024-07-31 18:55:47.108984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 18:55:47.292645: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-31 18:55:47.292671: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-31 18:55:47.996683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
data_folder = "../../data/STARMapPlus_nonrigid_simulation_different_distort/"

method = 'Spateo'
results_folder = "./nonrigid_different_distort_results/"+method
Path(results_folder).mkdir(parents=True, exist_ok=True)


In [3]:
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
i = 10
data_name = "well_{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s
j = 1
slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
repeat = 10

In [11]:
np.exp((5/ 5 * np.log(10)))

9.999999999999998

In [12]:
adata_pca = ad.concat([slice1,slice2], label='batch')
PCA(adata_pca, 'X', 'X_pca', 50, 'cuda')
slice1.obsm['X_pca'] = adata_pca[adata_pca.obs['batch'] == '0'].obsm['X_pca'].copy()
slice2.obsm['X_pca'] = adata_pca[adata_pca.obs['batch'] == '1'].obsm['X_pca'].copy()
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _ = st.align.morpho_align(
            models=[slice1.copy(), slice2.copy()],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            lambdaVF=1e1,
            beta=1e-1
            device='0',
            partial_robust_level=10,
            verbose=False,
            rep_layer=['X_pca'],
            rep_field=['obsm'],
            dissimilarity=['cos'],
        )
        
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial_nonrigid'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial_nonrigid'], align_slices[1].obsm['spatial']) / norm_scale**2
        print(f'------iteration: {l}----repeat: {k}-----mae: {mae}---------')
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + f"_{method}_differ_distort_{l}.npy"), alignment_results, allow_pickle=True)

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [2.4713s]
------iteration: 0----repeat: 0-----mae: 0.003508578699572048---------
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [2.4946s]
------iteration: 0----repeat: 1-----mae: 0.003143294889620258---------
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [2.4930s]
------iteration: 0----repeat: 2-----mae: 0.0034849971565095576---------
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [2.5165s]
------iteration: 0----repeat: 3-----mae: 0.0034149559908433135---------
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models ali

In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _, _ = paste_align_ref(
        models=[slice1.copy(), slice2.copy()],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            device="1",
            verbose=False,
            n_sampling=5000,
            sampling_method="random",
        )
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + "_SPACEL_differ_distort_{}.npy".format(l)), alignment_results, allow_pickle=True)

In [ ]:
# sagittal_morpho_results = {}
spatial_key="prealign_spatial"
key_added="align_spatial"
anno_key = 'Main_molecular_tissue_region'
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
# for i in range(16):
i = 3
# data_name = "well_{:02d}".format(i)
data_name = "sagittal{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s

for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))

    for l, no in enumerate(np.arange(0,0.22,0.02)):
        alignment_results = []
        for k in range(repeat):
            slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
            slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()

            sampling_num = 5000
            sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
            sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
            sub_slice1 = slice1[sampline_idx1,:]
            sub_slice2 = slice2[sampline_idx2,:]
    
            # align slices
            align_slices = spacel_align(
                models=[sub_slice1, sub_slice2],
                spatial_key=spatial_key,
                key_added=key_added,
                anno_key = anno_key,
            )
        
            mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
            mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
            alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j]+tps_slice_name[j+1]])
    
for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
    slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_0'].copy()

    # align slices
    align_slices = spacel_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        anno_key = anno_key,
    )

    mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
    mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
    alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j+1]+tps_slice_name[j]])

np.save(os.path.join(results_folder, data_name + "_SPACEL.npy"), alignment_results, allow_pickle=True)